![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


In [143]:
# Import necessary packages
import pandas as pd
import torch

from transformers import logging
logging.set_verbosity(logging.WARNING)

In [144]:
df = pd.read_csv('data/car_reviews.csv', delimiter=';')

df

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [145]:
df.shape

(5, 2)

In [146]:
review_list = df['Review'].tolist()
real_labels = [1 if label=='POSITIVE' else 0 for label in df['Class'].tolist()]

In [147]:
from transformers import pipeline

classifier = pipeline(task='sentiment-analysis')
predicted_labels = classifier(review_list)
predictions = [1 if predicted['label']=='POSITIVE' else 0 for predicted in predicted_labels]
print(predictions)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[1, 1, 1, 0, 1]


In [148]:
import evaluate

accuracy = evaluate.load('accuracy')
accuracy_result = accuracy.compute(predictions=predictions, references=real_labels)
accuracy_result = accuracy_result['accuracy']
print(accuracy_result)

f1 = evaluate.load('f1')
f1_result = f1.compute(predictions=predictions, references=real_labels)
f1_result = f1_result['f1']
print(f1_result)

0.8
0.8571428571428571


In [149]:
with open('data/reference_translations.txt', 'r') as file:
    lines = file.readlines()
references = [line.strip() for line in lines]
references

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.',
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']

In [150]:
frist_review = review_list[0]
frist_2_sentence = frist_review.split('.')[:2]
frist_2_sentence

['I am very satisfied with my 2014 Nissan NV SL',
 ' I use this van for my business deliveries and personal use']

In [151]:
en_to_es = pipeline(task='translation', model='Helsinki-NLP/opus-mt-en-es')
translated_review = en_to_es(frist_2_sentence)[0]['translation_text']

Device set to use cpu


In [152]:
bleu = evaluate.load('bleu')

bleu_score = bleu.compute(predictions=[translated_review], references=[references[0]])
bleu_score

{'bleu': 0.2635971381157267,
 'precisions': [1.0, 1.0, 1.0, 1.0],
 'brevity_penalty': 0.2635971381157267,
 'length_ratio': 0.42857142857142855,
 'translation_length': 9,
 'reference_length': 21}

In [153]:
context = review_list[1]
context

"The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found."

In [154]:
qa = pipeline(model='deepset/minilm-uncased-squad2')
question = 'What did he like about the brand?'
answer = qa(question=question, context=context)['answer']
answer

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


'ride quality, reliability'

In [155]:
last_review = review_list[-1]
last_review

"I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of lane changes. Th

In [156]:
summerizer = pipeline('summarization')
summarized_text = summerizer(last_review, min_length=50, max_length=55)[0]['summary_text']
summarized_text

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


' Nissan Rogue provides the desired SUV experience without burdening me with an exorbitant payment . Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more . The engine delivers strong performance, and'